# Disease prediction

In [1]:
# imports
import pandas as pd 
import numpy as np 

# 2.0 EDA

In [17]:
#reading the data in

df = pd.read_csv("Cardio-data/cardio_train.csv")

In [18]:
#data checking 

#df.info()
#df.describe()

df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
